# Imports 

In [59]:
# Imports
import numpy as np
import json
import glob
import pandas as pd

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy

#nltk
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

from sklearn.datasets import fetch_20newsgroups

## Lecture du fichier 

Transformation du JSON en DF exploitable

In [46]:
f = open("../../data/News_Category_Dataset_v2.json", "r")
Line = f.readline()
js = json.loads(Line)

df = pd.read_json(r'../../data/News_Category_Dataset_v2.json', lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [52]:
headlines_df = df.filter(["category", "headline", "short_description"], axis=1)
headlines_df.head()

,category,headline,short_description
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ..."


## Traitement des headlines

- Suppression des stop words
- lemmatization
- etc


In [56]:
headlines_df['headline'][0]

'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV'

### stop words

In [69]:
st_words = stopwords.words('english')
st_words.extend(['', ''])

In [74]:
def stopwords(sentence):

    list = []
    for word in sentence.split():
        if word not in st_words:
            print("Kept ==> " +word)
            list.append(word)
        else:
            print("Removed: " +word)
    return list

In [76]:
HL_no_sw = stopwords(headlines_df['headline'][0])

Kept ==> There
Kept ==> Were
Kept ==> 2
Kept ==> Mass
Kept ==> Shootings
Kept ==> In
Kept ==> Texas
Kept ==> Last
Kept ==> Week,
Kept ==> But
Kept ==> Only
Kept ==> 1
Kept ==> On
Kept ==> TV


In [77]:
print()

['There', 'Were', '2', 'Mass', 'Shootings', 'In', 'Texas', 'Last', 'Week,', 'But', 'Only', '1', 'On', 'TV']


In [30]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)